In [234]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from util import *

In [235]:
data_2009 = pd.read_csv("data/2009_grouped.csv")
data_2009.head()

,month,day_of_week,airline,origin,dest,num_of_flights,avg_delay
0,1,0,9E,ABE,DTW,8,3.875000
1,1,0,9E,ATL,AUS,7,-2.571429
2,1,0,9E,ATL,BHM,1,35.000000
3,1,0,9E,ATL,BNA,6,-4.833333
4,1,0,9E,ATL,BUF,4,7.750000


In [236]:
G_2009 = create_graph(data_2009)

In [263]:
G_2009.number_of_edges()

4619